In [1]:
# Adaptive histogram equalization
import cv2
import os, ntpath, re, shutil
from PIL import Image, ImageOps
import matplotlib.pyplot as plt

In [2]:
DATA_DIRECTORY = os.path.join('..', '..', 'data')

ORIGINAL_DATA = os.path.join(DATA_DIRECTORY, 'original', 'asl-alphabet')
FABRICATED_DATA = os.path.join(DATA_DIRECTORY, 'fabricated', 'asl_alphabet')


FABRICATED_TRAIN_DIR = os.path.join(FABRICATED_DATA, 'split_asl_alphabet_train')
FABRICATED_TEST_DIR = os.path.join(FABRICATED_DATA, 'split_asl_alphabet_test')

NEW_TRAIN_DIR = os.path.join(FABRICATED_DATA, 'processed_asl_train')

NEW_UNEDITED_TEST_DIR = os.path.join(FABRICATED_DATA, 'processed_original_test')
NEW_GRAYSCALE_TEST_DIR = os.path.join(FABRICATED_DATA, 'processed_grayscale_test')
NEW_GRAYSCALE_AHE_TEST_DIR = os.path.join(FABRICATED_DATA, 'processed_ahe_asl_test')
NEW_GRAYSCALE_AHE_NEG_TEST_DIR = os.path.join(FABRICATED_DATA, 'processed_ahe_inverted_asl_test')
NEW_RGB_AHE_TEST_DIR = os.path.join(FABRICATED_DATA, 'processed_rgb_ahe_test')
NEW_RGB_AHE_INVERTED_TEST_DIR = os.path.join(FABRICATED_DATA, 'processed_inverted_rgb_ahe_test')


# Used to quickly create all of these directories
FABRICATED_DIRS = [
    NEW_TRAIN_DIR,
    NEW_GRAYSCALE_AHE_TEST_DIR,
    NEW_GRAYSCALE_AHE_NEG_TEST_DIR,
    NEW_UNEDITED_TEST_DIR,
    NEW_GRAYSCALE_TEST_DIR,
    NEW_RGB_AHE_TEST_DIR,
    NEW_RGB_AHE_INVERTED_TEST_DIR
]

# Used to declaritively define relatinships of where we will be saving images
DIRECTORY_MAPPINGS = [
    # for the training directory, we just put all of the processed/original images in there
    {'orig_dirname': FABRICATED_TRAIN_DIR, 'new_dirnames': {'ahe': NEW_TRAIN_DIR, 'ahe_inverted': NEW_TRAIN_DIR, 'original': NEW_TRAIN_DIR, 'grayscale': NEW_TRAIN_DIR, 'unkown': NEW_TRAIN_DIR, 'inverted_unknown': NEW_TRAIN_DIR}},
    # We actually will create a new directory for each time of preprocessing for test images
    {'orig_dirname': FABRICATED_TEST_DIR, 'new_dirnames': {'ahe': NEW_GRAYSCALE_AHE_TEST_DIR, 'ahe_inverted': NEW_GRAYSCALE_AHE_NEG_TEST_DIR, 'original': NEW_UNEDITED_TEST_DIR, 'grayscale': NEW_GRAYSCALE_TEST_DIR, 'unkown': NEW_RGB_AHE_TEST_DIR, 'inverted_unknown': NEW_RGB_AHE_INVERTED_TEST_DIR}}
]

# A, B, C, D, DELETE, etc
ASL_FOLDERS = os.listdir(FABRICATED_TRAIN_DIR)

In [3]:
## Settings
TEST_RUN = False
DELETE_FOLDERS = True # Delete/Empty folders before starting

In [4]:
def delete_folders():
    for _dir in FABRICATED_DIRS:
        try:
            shutil.rmtree(_dir)
        except:
            # nothing to do here - it's likely that the directories just don't exist yet
            return

In [5]:
def create_folders_with_letter(letter):
    for _dir in FABRICATED_DIRS:
        dir_with_letter = os.path.join(_dir, letter)
        os.makedirs(dir_with_letter)

In [6]:
def create_folders():
    # create the output folders matching the train data set folders
    successfully_created = 0
    unsuccessfully_created = 0

    for sign_name in ASL_FOLDERS:   
        try:
            create_folders_with_letter(sign_name)
            successfully_created = successfully_created + 1
        except:
            unsuccessfully_created = unsuccessfully_created + 1


    print('successfully created {} sets of folders'.format(successfully_created))
    print('unsuccessfully created {} sets of folders (probably already existed)'.format(unsuccessfully_created))

In [7]:
def create_filename_for_fabricated_image(original_filename, enhancement):
    parts = re.split('\.', original_filename)
    file_body = '.'.join(parts[:-1])
    filename_extension = parts[-1]
    # the sticky part between the filename and the filename extension
    glue = enhancement
    return '{}_{}.{}'.format(file_body, glue, filename_extension)
create_filename_for_fabricated_image('A.jpg', 'reg')

'A_reg.jpg'

In [8]:
def create_ahe_image(img):
    # create a CLAHE object (Arguments are optional).
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4,4))
    ahe_img = clahe.apply(img)
    return Image.fromarray(ahe_img)

In [9]:
def create_inverse_image(img):
    return ImageOps.invert(img)

In [10]:
# need to determine what exactly this one is doing
def create_somerandomother_image(img):
    #-----Converting image to LAB Color model----------------------------------- 
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    #cv2.imshow("lab",lab)

    #-----Splitting the LAB image to different channels-------------------------
    l, a, b = cv2.split(lab)

    #-----Applying CLAHE to L-channel-------------------------------------------
    clahe = cv2.createCLAHE(clipLimit=15.0, tileGridSize=(10,10))
    cl = clahe.apply(l)
    #cv2.imshow('CLAHE output', cl)

    #-----Merge the CLAHE enhanced L-channel with the a and b channel-----------
    limg = cv2.merge((cl,a,b))
    #cv2.imshow('limg', limg)

    #-----Converting image from LAB Color model to RGB model--------------------
    final = cv2.cvtColor(limg, cv2.COLOR_LAB2RGB)
    return Image.fromarray(final)

In [11]:
def process_and_save_each_image_in_path(in_dir, sign_name, dir_mapping):
    for infile in os.listdir(in_dir):
        # Grayscale
        original_img = cv2.imread(os.path.join(in_dir, infile), 1)
        grayscale_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2GRAY)
        ahe_img = create_ahe_image(grayscale_img)
        invert_ahe_img = create_inverse_image(ahe_img)
        
        unkown_img = create_somerandomother_image(original_img)
        inverted_unknown = create_inverse_image(unkown_img)

        imgs_to_save = [Image.fromarray(original_img), Image.fromarray(grayscale_img), ahe_img, invert_ahe_img, unkown_img, inverted_unknown]
        enhancements = ['original', 'grayscale', 'ahe', 'ahe_inverted', 'unkown', 'inverted_unknown']
        #filenames = [create_filename_for_fabricated_image(infile, enhancement) for enhancement in enhancements]
        for img, enhancement in zip(imgs_to_save, enhancements):
            outdir_base = dir_mapping[enhancement]
            out_dir = os.path.join(outdir_base, sign_name)
            outfile_name = create_filename_for_fabricated_image(infile, enhancement)
            outfile_path = os.path.join(out_dir, outfile_name)

            img.save(outfile_path)
        if TEST_RUN:
            break

In [12]:
# create enhanced images using adaptive histogram equalization (ahe)
# create negative images of the ahe images
def enhance_and_save_images():
    if DELETE_FOLDERS:
        delete_folders()
    create_folders()

    # directory_settings Will be done for training and test image directories
    for directory_settings in DIRECTORY_MAPPINGS:
        from_dir = directory_settings['orig_dirname']
        print('copying from {}'.format(from_dir))
        dir_mapping = directory_settings['new_dirnames']
        for dirs in dir_mapping:
            print('-- copying to {}'.format(dir_mapping[dirs]))

        for sign_name in ASL_FOLDERS:
            in_dir = os.path.join(from_dir, sign_name)

            process_and_save_each_image_in_path(in_dir, sign_name, dir_mapping)
%time enhance_and_save_images()

successfully created 29 sets of folders
unsuccessfully created 0 sets of folders (probably already existed)
copying from ../../data/fabricated/asl_alphabet/split_asl_alphabet_train
-- copying to ../../data/fabricated/asl_alphabet/processed_asl_train
-- copying to ../../data/fabricated/asl_alphabet/processed_asl_train
-- copying to ../../data/fabricated/asl_alphabet/processed_asl_train
-- copying to ../../data/fabricated/asl_alphabet/processed_asl_train
-- copying to ../../data/fabricated/asl_alphabet/processed_asl_train
-- copying to ../../data/fabricated/asl_alphabet/processed_asl_train
copying from ../../data/fabricated/asl_alphabet/split_asl_alphabet_test
-- copying to ../../data/fabricated/asl_alphabet/processed_ahe_asl_test
-- copying to ../../data/fabricated/asl_alphabet/processed_ahe_inverted_asl_test
-- copying to ../../data/fabricated/asl_alphabet/processed_original_test
-- copying to ../../data/fabricated/asl_alphabet/processed_grayscale_test
-- copying to ../../data/fabricat